## Bayesian methods of hyperparameter optimization


In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.


In [1]:
#! pip install bayesian-optimization

In [2]:
import os
import warnings

warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [3]:
# List all files in a directory using os.listdir
os.listdir()

['Bayesian_optimization_case_study.ipynb',
 'flight_delays_test.csv.zip',
 'flight_delays_train.csv.zip']

## How does Bayesian optimization work?


Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.


<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />


## Let's look at a simple example


The first step is to create an optimizer. It uses two items:

- function to optimize
- bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. Smaller metrics are best. Hint: don't forget to use negative metric values.


Here we define our simple function we want to optimize.


In [4]:
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.


In [5]:
optimizer = BayesianOptimization(simple_func, {"a": (1, 3), "b": (4, 7)})

These are the main parameters of this function:

- **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

- **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.


Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.


In [6]:
optimizer.maximize(3, 2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 8.77      | 2.406     | 6.364     |
| 2         | 6.814     | 1.518     | 5.297     |
| 3         | 7.385     | 1.992     | 5.393     |
| 4         | 9.812     | 3.0       | 6.812     |
| 5         | 7.0       | 3.0       | 4.0       |


Great, now let's print the best parameters and the associated maximized target.


In [7]:
print(optimizer.max["params"])
optimizer.max["target"]

{'a': 3.0, 'b': 6.812245702960766}


9.812245702960766

## Test it on real data using the Light GBM


The dataset we will be working with is the famous flight departures dataset. Our modeling goal will be to predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset. As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.


**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [8]:
train_df = pd.read_csv("flight_delays_train.csv.zip")
test_df = pd.read_csv("flight_delays_test.csv.zip")

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**


In [9]:
train_df.head()


,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**


In [10]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
DepTime,100000.0,1341.52388,476.378445,1.0,931.0,1330.0,1733.0,2534.0
Distance,100000.0,729.39716,574.616860,30.0,317.0,575.0,957.0,4962.0


Notice, `DepTime` is the departure time in a numeric representation in 2400 hours.


**<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**


In [11]:
# train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df["dep_delayed_15min"].map({"Y": 1, "N": 0}).values

## Feature Engineering

Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.


In [12]:
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column


def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period
    return np.sin(value)


def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period
    return np.cos(value)


def feature_eng(df):
    df["flight"] = df["Origin"] + df["Dest"]
    df["Month"] = df.Month.map(lambda x: x.split("-")[-1]).astype("int32")
    df["DayofMonth"] = df.DayofMonth.map(lambda x: x.split("-")[-1]).astype("uint8")
    df["begin_of_month"] = (df["DayofMonth"] < 10).astype("uint8")
    df["midddle_of_month"] = (
        (df["DayofMonth"] >= 10) & (df["DayofMonth"] < 20)
    ).astype("uint8")
    df["end_of_month"] = (df["DayofMonth"] >= 20).astype("uint8")
    df["DayOfWeek"] = df.DayOfWeek.map(lambda x: x.split("-")[-1]).astype("uint8")
    df["hour"] = df.DepTime.map(lambda x: x / 100).astype("int32")
    df["morning"] = (
        df["hour"].map(lambda x: 1 if (x <= 11) & (x >= 7) else 0).astype("uint8")
    )
    df["day"] = (
        df["hour"].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype("uint8")
    )
    df["evening"] = (
        df["hour"].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype("uint8")
    )
    df["night"] = (
        df["hour"].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype("int32")
    )
    df["winter"] = df["Month"].map(lambda x: x in [12, 1, 2]).astype("int32")
    df["spring"] = df["Month"].map(lambda x: x in [3, 4, 5]).astype("int32")
    df["summer"] = df["Month"].map(lambda x: x in [6, 7, 8]).astype("int32")
    df["autumn"] = df["Month"].map(lambda x: x in [9, 10, 11]).astype("int32")
    df["holiday"] = (df["DayOfWeek"] >= 5).astype(int)
    df["weekday"] = (df["DayOfWeek"] < 5).astype(int)
    df["airport_dest_per_month"] = df.groupby(["Dest", "Month"])["Dest"].transform(
        "count"
    )
    df["airport_origin_per_month"] = df.groupby(["Origin", "Month"])[
        "Origin"
    ].transform("count")
    df["airport_dest_count"] = df.groupby(["Dest"])["Dest"].transform("count")
    df["airport_origin_count"] = df.groupby(["Origin"])["Origin"].transform("count")
    df["carrier_count"] = df.groupby(["UniqueCarrier"])["Dest"].transform("count")
    df["carrier_count_per month"] = df.groupby(["UniqueCarrier", "Month"])[
        "Dest"
    ].transform("count")
    df["deptime_cos"] = df["DepTime"].map(make_harmonic_features_cos)
    df["deptime_sin"] = df["DepTime"].map(make_harmonic_features_sin)
    df["flightUC"] = df["flight"] + df["UniqueCarrier"]
    df["DestUC"] = df["Dest"] + df["UniqueCarrier"]
    df["OriginUC"] = df["Origin"] + df["UniqueCarrier"]
    return df.drop("DepTime", axis=1)

Concatenate the training and testing dataframes.


In [13]:
full_df = pd.concat([train_df.drop("dep_delayed_15min", axis=1), test_df])
full_df = feature_eng(full_df)

Apply the earlier defined feature engineering functions to the full dataframe.


In [14]:
for column in [
    "UniqueCarrier",
    "Origin",
    "Dest",
    "flight",
    "flightUC",
    "DestUC",
    "OriginUC",
]:
    full_df[column] = label_enc(full_df[column])

Split the new full dataframe into X_train and X_test.


In [15]:
X_train = full_df[: train_df.shape[0]]
X_test = full_df[train_df.shape[0] :]

Create a list of the categorical features.


In [16]:
categorical_features = [
    "Month",
    "DayOfWeek",
    "UniqueCarrier",
    "Origin",
    "Dest",
    "flight",
    "flightUC",
    "DestUC",
    "OriginUC",
]

Let's build a light GBM model to test the bayesian optimizer.


### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

- Faster training speed and higher efficiency.
- Lower memory usage.
- Better accuracy.
- Support of parallel and GPU learning.
- Capable of handling large-scale data.


First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.


In [17]:
def lgb_eval(
    num_leaves, max_depth, lambda_l2, lambda_l1, min_child_samples, min_data_in_leaf
):
    params = {
        "objective": "binary",
        "metric": "auc",
        "is_unbalance": True,
        "num_leaves": int(num_leaves),
        "max_depth": int(max_depth),
        "lambda_l2": lambda_l2,
        "lambda_l1": lambda_l1,
        "num_threads": 20,
        "min_child_samples": int(min_child_samples),
        "min_data_in_leaf": int(min_data_in_leaf),
        "learning_rate": 0.03,
        "subsample_freq": 5,
        "bagging_seed": 42,
        "verbosity": -1,
    }
    lgtrain = lightgbm.Dataset(
        X_train, y_train, categorical_feature=categorical_features
    )
    cv_result = lightgbm.cv(
        params, lgtrain, 1000, early_stopping_rounds=100, stratified=True, nfold=3
    )
    return cv_result["auc-mean"][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [18]:
lgbBO = BayesianOptimization(
    lgb_eval,
    {
        "num_leaves": (25, 4000),
        "max_depth": (5, 63),
        "lambda_l2": (0.0, 0.05),
        "lambda_l1": (0.0, 0.05),
        "min_child_samples": (50, 1000),
        "min_data_in_leaf": (100, 2000),
    },
)

lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
[LightGBM] [Warning] min_data_in_leaf is set=1006, min_child_samples=529 will be ignored. Current value: min_data_in_leaf=1006
| 1         | 0.7335    | 0.04309   | 0.005191  | 53.61     | 530.0     | 1.007e+03 | 160.8     |
| 2         | 0.7433    | 0.01598   | 0.01444   | 59.1      | 421.5     | 1.694e+03 | 3.494e+03 |
| 3         | 0.7429    | 0.04692   | 0.01904   | 16.1      | 701.9     | 1.238e+03 | 2.077e+03 |
| 4         | 0.7439    | 0.01739   | 0.02822   | 12.49     | 571.7     | 1.939e+03 | 994.4     |
| 5         | 0.7433    | 0.0254    | 0.03595   | 15.64     | 50.36     | 1.781e+03 | 1.58e+03  |
| 6         | 0.7432    | 0.03968   | 0.02263   | 33.39     | 245.8     | 1.461e+03 | 2.479e+03 |
| 7         | 0.7434    | 0.01444   | 0.03585   | 55.04     | 415.7     | 1.938e+03 | 1.0

**<font color='teal'> Print the best result by using the '.max' function.</font>**


In [19]:
lgbBO.max

{'target': 0.7439109967620198,
 'params': {'lambda_l1': 0.017386232397267744,
  'lambda_l2': 0.028219468466145738,
  'max_depth': 12.492627634789061,
  'min_child_samples': 571.7216512656624,
  'min_data_in_leaf': 1939.103490337107,
  'num_leaves': 994.4419725168341}}

In [20]:
print(f"Max AUC param values:\n{lgbBO.max['params']}")
print(f"Max AUC Score:\n{lgbBO.max['target']:.4f}") 

Max AUC param values:
{'lambda_l1': 0.017386232397267744, 'lambda_l2': 0.028219468466145738, 'max_depth': 12.492627634789061, 'min_child_samples': 571.7216512656624, 'min_data_in_leaf': 1939.103490337107, 'num_leaves': 994.4419725168341}
Max AUC Score:
0.7439


Review the process at each step by using the '.res[0]' function.


In [21]:
lgbBO.res[0]

{'target': 0.7334606840587,
 'params': {'lambda_l1': 0.04308867286403459,
  'lambda_l2': 0.005190522763888839,
  'max_depth': 53.61050663545995,
  'min_child_samples': 529.9791235785499,
  'min_data_in_leaf': 1006.7439078257514,
  'num_leaves': 160.8120322435672}}

In [22]:

targets = [lgbBO.res[i]["target"] for i in range(len(lgbBO.res))]
df = pd.DataFrame()
for i in range(len(lgbBO.res)):
    df = pd.concat([df, pd.DataFrame(lgbBO.res[i]["params"], index=[i])])

df["target"] = targets
# display dataframe of the results
display(df.sort_values("target", ascending=False))

# get the param values with the best score
max_params = df.sort_values("target", ascending=False).iloc[0, :-1].to_dict()

# ensure interger columns have integer type values
interger_columns = ["num_leaves", "max_depth", "min_child_samples", "min_data_in_leaf"]
for col in interger_columns:
    max_params[col] = int(max_params[col])

display(max_params)

,lambda_l1,lambda_l2,max_depth,min_child_samples,min_data_in_leaf,num_leaves,target
3,0.017386,0.028219,12.492628,571.721651,1939.103490,994.441973,0.743911
9,0.019020,0.028699,35.012851,995.593673,1979.814942,2994.882389,0.743774
10,0.003088,0.011664,21.527088,989.423573,1999.603325,3947.078669,0.743717
7,0.036010,0.046351,50.903036,359.350646,1917.209085,2242.347262,0.743569
6,0.014436,0.035848,55.043149,415.677274,1938.135280,1026.450761,0.743351
4,0.025399,0.035954,15.641193,50.361626,1780.894661,1580.383752,0.743331
1,0.015980,0.014439,59.103980,421.454558,1694.023997,3493.805245,0.743294
5,0.039676,0.022627,33.394772,245.768917,1461.357264,2479.170030,0.743234
2,0.046924,0.019037,16.095897,701.913841,1237.979882,2077.120100,0.742938
0,0.043089,0.005191,53.610507,529.979124,1006.743908,160.812032,0.733461


{'lambda_l1': 0.017386232397267744,
 'lambda_l2': 0.028219468466145738,
 'max_depth': 12,
 'min_child_samples': 571,
 'min_data_in_leaf': 1939,
 'num_leaves': 994}

In [23]:
base_params = {
    "objective": "binary",
    "metric": "auc",
    "is_unbalance": True,
    "verbosity": -1,

}

max_params |= base_params
max_params

{'lambda_l1': 0.017386232397267744,
 'lambda_l2': 0.028219468466145738,
 'max_depth': 12,
 'min_child_samples': 571,
 'min_data_in_leaf': 1939,
 'num_leaves': 994,
 'objective': 'binary',
 'metric': 'auc',
 'is_unbalance': True,
 'verbosity': -1}

In [24]:

lgtrain = lightgbm.Dataset(X_train, y_train, categorical_feature=categorical_features)

results = lightgbm.cv(
    max_params, lgtrain, 1000, early_stopping_rounds=100, stratified=True, nfold=5
)

In [25]:
display(len(results["auc-mean"]))
results["auc-mean"][-1].round(3)

376

0.746